In [46]:
# importing libraries
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
# reading data with house pricing
train = pd.read_csv('Data/train.csv', index_col='Id')
test = pd.read_csv('Data/test.csv', index_col='Id') # used at the end to evaluation model efficiency

In [59]:
# examining data
# print('SalePrice' in train.columns, 'SalePrice' in test.columns, sep='\n')
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [67]:
# setting 'Price' column as target for predctions
y = train.SalePrice

#features preparation
features = train.drop(['SalePrice'], axis=1) # remove 'SalePrice' column

# only numerical predicators
X = features.select_dtypes(exclude=['object'])

# only numerical predicators from test dataset
X_test_full = test.select_dtypes(exclude=['object'])

# divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [68]:
# define function for column imputation (missing values)
def imputation_dataset(X_train, X_valid, X_test):
    imputer = SimpleImputer()
    
    imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
    imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))
    imputed_X_test = pd.DataFrame(imputer.transform(X_test))
    
    imputed_X_train.columns = X_train.columns
    imputed_X_valid.columns = X_valid.columns
    imputed_X_test.columns = X_test.columns
    
    return [imputed_X_train, imputed_X_valid, imputed_X_test]

In [69]:
# imputing columns with missing values
imp_X_train, imp_X_valid, imp_X_test = imputation_dataset(X_train_full, X_valid_full, X_test_full)

In [ ]:
# retrieves categorical columns from 'train', 'valid' and 'test' subests
# cat_col = [col for col in train.columns if train[col].dtype == 'object'

In [81]:
# retrieves categorical columns with missing values and drops them
cat_cols = [col for col in train.columns if 
                        train[col].dtype == 'object' and 
                        not train[col].isnull().any()] 
X_train_proc = pd.concat([imp_X_train, train[cat_cols]], axis=1)
X_train_proc.to_csv('Data/X_train_proc.csv')

In [82]:
X_train_proc.shape

(1461, 63)

In [44]:
# evaluating which columns has missing values (only from numerical features)
cols_with_missing = [col for col in X_train_full if X_train_full[col].isnull().any()]
cols_with_missing

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [13]:
# drop columns with missing values from training and test datasets
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

In [22]:
X_train_full.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


### Evaluating cardinality in categorical values

In [17]:
# retrieving columns with low cardinality
low_cardinality_cols = [col for col in X_train_full.columns if 
                        X_train_full[col].nunique() < 10 and
                        X_train_full[col].dtype == "object"]
low_cardinality_cols

[]

In [18]:
#
num_cols = [col for col in X_train_full if X_train_full[col].dtype in 
            ['int64', 'float64']]
num_cols

9

In [19]:
correct_cols = low_cardinality_cols + num_cols
correct_cols

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount']